# Working with Text

In [3]:
from importlib.metadata import version
import re

print("Torch version:", version("torch"))
print("Tiktoken version:", version("tiktoken"))

Torch version: 2.3.0
Tiktoken version: 0.8.0


## 1.1 Data Loading

In [4]:
with open("../../data/raw/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Length of raw text: ", len(raw_text))

Length of raw text:  20479


## 1.2 Tokenization

In [5]:
text  = "Hello, world. This, is a test."

result = re.split(r"\W+", text)
print("Tokens: ", result)

result = re.split(r'([,.]|\s)', text)
print("Tokens: ", result)

Tokens:  ['Hello', 'world', 'This', 'is', 'a', 'test', '']
Tokens:  ['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
[item for item in result if item.strip()]

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

### 1.2.1 Apply tokenization to the raw text

In [7]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:10])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ']


In [8]:
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


#### The total number of tokens

In [9]:
len(preprocessed)

4690

## 1.3 Convert tokens into token IDs

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print("Vocabulary size: ", vocab_size)

Vocabulary size:  1130


In [11]:
vocab = {token:integer for integer, token in enumerate(all_words)}

list(vocab.items())[:200]

[('!', 0),
 ('"', 1),
 ("'", 2),
 ('(', 3),
 (')', 4),
 (',', 5),
 ('--', 6),
 ('.', 7),
 (':', 8),
 (';', 9),
 ('?', 10),
 ('A', 11),
 ('Ah', 12),
 ('Among', 13),
 ('And', 14),
 ('Are', 15),
 ('Arrt', 16),
 ('As', 17),
 ('At', 18),
 ('Be', 19),
 ('Begin', 20),
 ('Burlington', 21),
 ('But', 22),
 ('By', 23),
 ('Carlo', 24),
 ('Chicago', 25),
 ('Claude', 26),
 ('Come', 27),
 ('Croft', 28),
 ('Destroyed', 29),
 ('Devonshire', 30),
 ('Don', 31),
 ('Dubarry', 32),
 ('Emperors', 33),
 ('Florence', 34),
 ('For', 35),
 ('Gallery', 36),
 ('Gideon', 37),
 ('Gisburn', 38),
 ('Gisburns', 39),
 ('Grafton', 40),
 ('Greek', 41),
 ('Grindle', 42),
 ('Grindles', 43),
 ('HAD', 44),
 ('Had', 45),
 ('Hang', 46),
 ('Has', 47),
 ('He', 48),
 ('Her', 49),
 ('Hermia', 50),
 ('His', 51),
 ('How', 52),
 ('I', 53),
 ('If', 54),
 ('In', 55),
 ('It', 56),
 ('Jack', 57),
 ('Jove', 58),
 ('Just', 59),
 ('Lord', 60),
 ('Made', 61),
 ('Miss', 62),
 ('Money', 63),
 ('Monte', 64),
 ('Moon-dancers', 65),
 ('Mr', 66),
 (

### 1.3.1 Implement encode and decode 
- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

In [12]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_init = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_init[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
        "Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know,"" Mrs. Gisburn said with pardonable pride.'

## 1.4 Adding special tokens

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_words.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_words)}

In [16]:
len(vocab.items())

1132

In [17]:
list(vocab.items())[-3:]

[('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]

In [18]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_init = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_init
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_init[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [30]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text3 = "Dhaka Gazipur  Amtoli"

text = " <|endotext|> ".join((text1, text2, text3))
print(text)

Hello, do you like tea? <|endotext|> In the sunlit terraces of the palace. <|endotext|> Dhaka Gazipur  Amtoli


In [31]:
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1131, 7, 1131, 1131, 1131, 1131]
